In [ ]:
import http.client
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
import datetime as dt
import numpy as np

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.cloud import bigquery
project_id = ''
client = bigquery.Client(project=project_id)

In [ ]:
df = client.query('''
select distinct EXTRACT(MONTH from block_timestamp) as block_timestamp
  , tt.from_address
from `nansen-query.raw_arbitrum.transactions` tt
inner join (select from_address, 
                   min(block_timestamp)
          from `nansen-query.raw_arbitrum.transactions`
          group by from_address
          having min(block_timestamp)>='2022-01-01') trans
          on tt.from_address = trans.from_address
 ''').to_dataframe()

In [ ]:
df.columns = ['block_timestamp','from_address']

In [ ]:
df = df[df['block_timestamp']!='10']

In [ ]:
df['block_timestamp'] = df['block_timestamp'].astype('int')

In [ ]:
df['block_timestamp'] = np.where(df['block_timestamp']=='1','2022-01-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='2','2022-02-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='3','2022-03-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='4','2022-04-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='5','2022-05-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='6','2022-06-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='7','2022-07-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='8','2022-08-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']=='9','2022-09-01',df['block_timestamp'])
df['block_timestamp'] = np.where(df['block_timestamp']==1,'2022-01-01',df['block_timestamp'])

In [ ]:
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'],format='%Y-%m-%d')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['tx_month'] = df['block_timestamp'].apply(lambda x: dt.date(x.year, x.month,1))

df['acq_month'] = df.groupby('from_address')['tx_month'].transform('min')

df.loc[df['tx_month'] != df['acq_month']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,block_timestamp,from_address,tx_month,acq_month
1,2022-02-01,0xd20e1ad727f546e89ba07fb781d559f2e420be24,2022-02-01,2022-01-01
5,2022-02-01,0x307ab299a19349c15d21708d8f75e6b6b52b8ecb,2022-02-01,2022-01-01
8,2022-02-01,0x1e61147584b756c042a00aa044cd1e70dca853f6,2022-02-01,2022-01-01
10,2022-02-01,0xf76f10be11a9e2b313a7827d3a8e72147cc7b6dc,2022-02-01,2022-01-01
12,2022-02-01,0x21a6f46fe65952d1506b4f5782bff8671e664800,2022-02-01,2022-01-01


In [ ]:
def diff_month(x):

    d1 = x['tx_month']

    d2 = x["acq_month"]

    return ((d1.year - d2.year) * 12 + d1.month - d2.month)+1


df['cohort_idx'] = df.apply(lambda x: diff_month(x), axis=1)

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,block_timestamp,from_address,tx_month,acq_month,cohort_idx
0,2022-02-01,0x3ff96b1fa67091fbf5075dc90ecf299f9abb6f3c,2022-02-01,2022-02-01,1
1,2022-02-01,0xd20e1ad727f546e89ba07fb781d559f2e420be24,2022-02-01,2022-01-01,2
2,2022-02-01,0x031b87fbfac699a2083bd10ca2fb8492a1cc40af,2022-02-01,2022-02-01,1
3,2022-02-01,0xdf27b7509daa8e189f49b2c1118377661e2ff619,2022-02-01,2022-02-01,1
4,2022-02-01,0xc206121caf283f2440e26da0adb3967fbc3b79db,2022-02-01,2022-02-01,1


In [ ]:
def get_cohort_matrix(data, var='from_address', fun=pd.Series.nunique):

    cd = data.groupby(['acq_month', 'cohort_idx'])[var].apply(fun).reset_index()

    cc = cd.pivot_table(index = 'acq_month',

                        columns = 'cohort_idx',

                        values = var)

    cs = cc.iloc[:,0]

    retention = cc.divide(cs, axis = 0)

    retention = retention.round(3) * 100

    return cc, retention

In [ ]:
cc, retention = get_cohort_matrix(df)

cc.to_csv('arb_retention.csv')

In [ ]:
cc

cohort_idx,1,2,3,4,5,6,7,8,9
acq_month,,,,,,,,,
2022-01-01,30779.0,12904.0,9784.0,8425.0,7354.0,6682.0,5237.0,5517.0,5622.0
2022-02-01,40341.0,18935.0,12067.0,9767.0,8864.0,6454.0,6635.0,6870.0,NaN
2022-03-01,57632.0,16961.0,12453.0,11822.0,9271.0,7504.0,6982.0,NaN,NaN
2022-04-01,96369.0,29440.0,23280.0,16439.0,17055.0,16191.0,NaN,NaN,NaN
2022-05-01,93293.0,33570.0,24408.0,20672.0,19039.0,NaN,NaN,NaN,NaN
2022-06-01,183191.0,70688.0,50840.0,45331.0,NaN,NaN,NaN,NaN,NaN
2022-07-01,128201.0,26414.0,30514.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,132369.0,45216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-01,180766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
